In [1]:
#3Company_combos_Quantity

import pandas as pd
from itertools import combinations

# ==========================================================
#                      USER INPUTS
# ==========================================================

input_file = input("\nEnter input Excel file path: ").strip()
output_file = input("Enter output Excel file path: ").strip()

def get_float(prompt):
    while True:
        try:
            return float(input(prompt).strip())
        except:
            print("Invalid value! Please enter a numeric value.")

def get_int(prompt):
    while True:
        try:
            return int(input(prompt).strip())
        except:
            print("Invalid value! Please enter an integer.")

INVESTMENT = get_float("\nEnter investment amount per company (example: 50000): ")
COMBO_SIZE = get_int("Enter number of companies per combination (example: 4): ")
OBSERVATION_DAYS = get_int("Enter number of observation days (example: 45): ")
MONTH_ROLLING = get_int("Enter number of months to skip before starting: ")

# Rolling type (year/month/week/day)
while True:
    ROLL_TYPE = input("Enter rolling type (year/month/week/day): ").strip().lower()
    if ROLL_TYPE in ["year", "month", "week", "day"]:
        break
    print("Invalid type! Enter year, month, week, or day.")

INCREMENT = get_int("Enter rolling increment (example: 1): ")

# ==========================================================
#                     LOAD DATA
# ==========================================================

print("\nLoading file...")
df = pd.read_excel(input_file)
df.columns = df.columns.str.strip()

# Detect date column
date_col_candidates = [c for c in df.columns if "date" in c.lower()]
if not date_col_candidates:
    raise ValueError("DATE column not found!")

DATE_COL = date_col_candidates[0]
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce", dayfirst=True)
df = df.dropna(subset=[DATE_COL]).sort_values(DATE_COL)

# Detect numeric company columns
company_names = [col for col in df.columns if col != DATE_COL and pd.api.types.is_numeric_dtype(df[col])]
if not company_names:
    raise ValueError("No numeric company columns found!")

print(f"\nDetected {len(company_names)} companies")

# ==========================================================
#     START DATE AFTER SKIPPING FIRST X MONTHS
# ==========================================================

start_date = df[DATE_COL].min() + pd.DateOffset(months=MONTH_ROLLING)
current = start_date.replace(day=1)

print(f"\nDataset start: {df[DATE_COL].min().date()}")
print(f"Processing begins after {MONTH_ROLLING} months: {current.date()}")
print(f"Rolling type: {ROLL_TYPE} | Increment: {INCREMENT}")

# ==========================================================
#     FUNCTION: ROLLING STEP CALCULATION
# ==========================================================

def get_next_date(current):
    if ROLL_TYPE == "year":
        return current + pd.DateOffset(years=INCREMENT)
    elif ROLL_TYPE == "month":
        return current + pd.DateOffset(months=INCREMENT)
    elif ROLL_TYPE == "week":
        return current + pd.DateOffset(weeks=INCREMENT)
    elif ROLL_TYPE == "day":
        return current + pd.DateOffset(days=INCREMENT)

# ==========================================================
#             WRITE MULTIPLE SHEETS 
# ==========================================================

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

    last_date = df[DATE_COL].max()
    sheet_counter = 1  

    while current <= last_date:

        observe_start = current
        observe_end = current + pd.Timedelta(days=OBSERVATION_DAYS)

        data = df[(df[DATE_COL] >= observe_start) & (df[DATE_COL] <= observe_end)]

        if data.empty:
            current = get_next_date(current)
            continue

        selected_date = data[DATE_COL].min()
        row = data[data[DATE_COL] == selected_date].iloc[0]

        print(f"\nSheet: Week{sheet_counter} | Using date: {selected_date.date()}")

        combos_output = [
            [
                ",".join(combo),
                ",".join(
                    "NA" if pd.isna(row[c]) or row[c] == 0 else f"{INVESTMENT / row[c]:.2f}"
                    for c in combo
                ),
                selected_date
            ]
            for combo in combinations(company_names, COMBO_SIZE)
        ]

        out_df = pd.DataFrame(combos_output, columns=["Combo", "Quantities", "Calculation_Date"])

        sheet_name = f"Week{sheet_counter}"
        out_df.to_excel(writer, sheet_name=sheet_name, index=False)

        sheet_counter += 1
        current = get_next_date(current)

print("\nPROCESS COMPLETED SUCCESSFULLY!")
print("Saved at:", output_file)



Enter input Excel file path: E:\3Combinations_feb24_to_jan26\NIFTY50_CLOSE_MASTER.xlsx
Enter output Excel file path: E:\3Combinations_feb24_to_jan26\3Company_combos_Quantity.xlsx

Enter investment amount per company (example: 50000): 50000
Enter number of companies per combination (example: 4): 3
Enter number of observation days (example: 45): 45
Enter number of months to skip before starting: 3
Enter rolling type (year/month/week/day): week
Enter rolling increment (example: 1): 1

Loading file...

Detected 50 companies

Dataset start: 2024-02-14
Processing begins after 3 months: 2024-05-01
Rolling type: week | Increment: 1

Sheet: Week1 | Using date: 2024-05-02

Sheet: Week2 | Using date: 2024-05-08

Sheet: Week3 | Using date: 2024-05-15

Sheet: Week4 | Using date: 2024-05-22

Sheet: Week5 | Using date: 2024-05-29

Sheet: Week6 | Using date: 2024-06-05

Sheet: Week7 | Using date: 2024-06-12

Sheet: Week8 | Using date: 2024-06-19

Sheet: Week9 | Using date: 2024-06-26

Sheet: Week10 |

In [2]:
# 3Combo_TotalPrices

import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
import os

# ======================================================
#               USER INPUTS 
# ======================================================

file_prices = input("\nEnter price file path: ").strip()
file_combos = input("Enter combo quantity file path: ").strip()
output_folder = input("Enter output folder path: ").strip()

while True:
    try:
        OBS_DAYS = int(input("\nEnter observation days (example: 45): ").strip())
        break
    except:
        print("Invalid input! Enter a number.")

# ---------------- Rolling Frequency Input ----------------
valid_roll_types = ["day", "week", "month", "year"]

while True:
    ROLL_TYPE = input("\nEnter rolling type (day/week/month/year): ").strip().lower()
    if ROLL_TYPE in valid_roll_types:
        break
    print("Invalid! Choose from day / week / month / year")

while True:
    try:
        ROLL_VALUE = int(input("Enter rolling increment : ").strip())
        break
    except:
        print("Invalid number! Try again.")

# Make output folder
os.makedirs(output_folder, exist_ok=True)

# ======================================================
#                LOAD PRICE DATA
# ======================================================

df_price = pd.read_excel(file_prices)
date_col = df_price.columns[0]
company_cols = df_price.columns[1:]

df_price[date_col] = pd.to_datetime(df_price[date_col], dayfirst=True)
df_price = df_price.sort_values(date_col)

# Skip 3 months
first_date = df_price[date_col].min()
skip_date = first_date + pd.DateOffset(months=3)
df_filtered = df_price[df_price[date_col] > skip_date].copy()

print("\nDataset start:", first_date.date())
print("After skipping first 3 months:", skip_date.date())

# ======================================================
#                LOAD COMBO DATA
# ======================================================

df_combo = pd.read_excel(file_combos)
df_combo.columns = df_combo.columns.str.strip()

df_combo[['C1','C2','C3']] = df_combo['Combo'].str.split(',', expand=True)

def split_qty(q):
    arr = []
    for x in str(q).split(","):
        try:
            arr.append(float(x.strip()))
        except:
            arr.append(0.0)
    # Ensure 4 elements
    while len(arr) < 3:
        arr.append(0.0)
    return arr[:3]

df_combo[['Q1','Q2','Q3']] = df_combo['Quantities'].apply(split_qty).tolist()

# Pre-map company index
col_index = {col: i for i, col in enumerate(company_cols)}

combo_indices = df_combo[['C1','C2','C3']].applymap(
    lambda x: col_index.get(x, -1)
).values.astype(int)

qty_matrix = df_combo[['Q1','Q2','Q3']].values.astype(float)

# ======================================================
#         HELPER FUNCTION FOR ROLLING DATE OFFSET
# ======================================================

def shift_date(start, roll_type, value):
    """Return shifted date by selected rolling frequency."""
    if roll_type == "day":
        return start + timedelta(days=value)
    elif roll_type == "week":
        return start + timedelta(weeks=value)
    elif roll_type == "month":
        return start + pd.DateOffset(months=value)
    elif roll_type == "year":
        return start + pd.DateOffset(years=value)

# ======================================================
#               MAIN ROLLING WINDOW LOOP
# ======================================================

current_start = df_filtered[date_col].min()
window_count = 1

while current_start <= df_filtered[date_col].max():

    window_end = current_start + timedelta(days=OBS_DAYS - 1)

    df_window = df_filtered[
        (df_filtered[date_col] >= current_start) &
        (df_filtered[date_col] <= window_end)
    ].copy()

    if df_window.empty:
        break

    print(f"\nProcessing Window {window_count}")
    print("Start:", df_window[date_col].min().date())
    print("End  :", df_window[date_col].max().date())

    # ---------- Fast Calculation Section ----------
    results = []
    df_prices_np = df_window[company_cols].to_numpy(float)

    total_rows = len(df_window) * len(df_combo)
    pbar = tqdm(total=total_rows, desc=f"Window {window_count}", unit="rows")

    for idx, dp_row in enumerate(df_prices_np):
        date_value = df_window[date_col].iloc[idx]

        # Multiply entire combo price matrix at once
        price_matrix = dp_row[combo_indices]  
        price_matrix = np.where(combo_indices == -1, 0, price_matrix)

        total_prices = np.sum(price_matrix * qty_matrix, axis=1)

        # Create output rows fast
        for i in range(len(df_combo)):
            results.append([
                date_value,
                df_combo["Combo"].iloc[i],
                list(price_matrix[i]),
                total_prices[i]
            ])
            pbar.update(1)

    pbar.close()

    df_result = pd.DataFrame(results, columns=[
        "DATE", "Combo", "Price_Combo", "Total_Price"
    ])

    output_csv = os.path.join(
        output_folder,
        f"Window_{window_count}_{OBS_DAYS}Days.csv"
    )
    df_result.to_csv(output_csv, index=False)
    print("Saved:", output_csv)

    # ---------- Shift by User-defined Rolling Type ----------
    current_start = shift_date(current_start, ROLL_TYPE, ROLL_VALUE)
    window_count += 1

print("\nAll rolling CSV files generated successfully!")



Enter price file path: E:\3Combinations_feb24_to_jan26\NIFTY50_CLOSE_MASTER.xlsx
Enter combo quantity file path: E:\3Combinations_feb24_to_jan26\3Company_combos_Quantity.xlsx
Enter output folder path: E:\3Combinations_feb24_to_jan26\3_company_Total_Price

Enter observation days (example: 45): 45

Enter rolling type (day/week/month/year): week
Enter rolling increment : 
Invalid number! Try again.
Enter rolling increment : 1

Dataset start: 2024-02-14
After skipping first 3 months: 2024-05-14


C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_11640\3187331916.py:88: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combo_indices = df_combo[['C1','C2','C3']].applymap(



Processing Window 1
Start: 2024-05-15
End  : 2024-06-28


Window 1: 100%|███████████████████████████████████████████████████████████| 607600/607600 [00:09<00:00, 61290.23rows/s]


Saved: E:\3Combinations_feb24_to_jan26\3_company_Total_Price\Window_1_45Days.csv

Processing Window 2
Start: 2024-05-22
End  : 2024-07-05


...

Saved: E:\3Combinations_feb24_to_jan26\3_company_Total_Price\Window_84_45Days.csv

Processing Window 85
Start: 2025-12-24
End  : 2026-01-02


Window 85: 100%|██████████████████████████████████████████████████████████| 156800/156800 [00:02<00:00, 71474.57rows/s]


Saved: E:\3Combinations_feb24_to_jan26\3_company_Total_Price\Window_85_45Days.csv

Processing Window 86
Start: 2025-12-31
End  : 2026-01-02


Window 86: 100%|████████████████████████████████████████████████████████████| 58800/58800 [00:00<00:00, 71798.13rows/s]


Saved: E:\3Combinations_feb24_to_jan26\3_company_Total_Price\Window_86_45Days.csv

All rolling CSV files generated successfully!


In [3]:
#3Company_EURINR_Ratio_50DMA

import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
import os
from collections import deque, defaultdict

# ====================== REQUIRED USER INPUTS ======================
INPUT_PRICE_FOLDER = input("Enter input PRICE folder path: ").strip()
INPUT_FX_FILE = input("Enter FX (EURINR) file path: ").strip()
OUTPUT_FOLDER = input("Enter OUTPUT folder path: ").strip()
MOVING_AVG_DAYS = int(input("Enter Moving Average days : "))

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====================== LOAD FX DATA ======================
fx = pd.read_excel(INPUT_FX_FILE, usecols=["DATE", "EURINR"])
fx["DATE"] = pd.to_datetime(fx["DATE"])
fx.set_index("DATE", inplace=True)

# ====================== GET ALL INPUT FILES ======================
csv_files = sorted(glob.glob(os.path.join(INPUT_PRICE_FOLDER, "*.csv")))
print(f"Total input CSV files detected: {len(csv_files)}")

# ====================== PROCESS EACH FILE ======================
for file_csv in csv_files:

    print(f"\nProcessing: {os.path.basename(file_csv)}")

    output_file = os.path.join(
        OUTPUT_FOLDER,
        f"{MOVING_AVG_DAYS}DMA_{os.path.basename(file_csv)}"
    )

    # Deque and cumulative sum per Combo
    ratio_queue = defaultdict(lambda: deque(maxlen=MOVING_AVG_DAYS))
    ratio_sum = defaultdict(float)

    header_written = False

    for chunk in tqdm(
        pd.read_csv(
            file_csv,
            chunksize=50000,
            parse_dates=["DATE"],
            usecols=["DATE", "Combo", "Total_Price"]
        ),
        desc="Streaming",
        unit=" rows"
    ):
        # Merge EURINR data
        chunk = chunk.join(fx, on="DATE", how="left")
        chunk.dropna(subset=["EURINR"], inplace=True)

        if chunk.empty:
            continue

        # Compute EURINR Ratio
        eur_ratio = chunk["Total_Price"].values / chunk["EURINR"].values
        dma_values = np.zeros(len(chunk))
        combos = chunk["Combo"].values

        # Rolling moving average calculation
        for i, combo in enumerate(combos):
            q = ratio_queue[combo]

            # Maintain rolling sum
            if len(q) == MOVING_AVG_DAYS:
                ratio_sum[combo] -= q[0]

            q.append(eur_ratio[i])
            ratio_sum[combo] += eur_ratio[i]

            dma_values[i] = ratio_sum[combo] / len(q)

        # Save chunk to output
        out = pd.DataFrame({
            "DATE": chunk["DATE"].values,
            "Combo": combos,
            "EURINR_Ratio": eur_ratio,
            f"{MOVING_AVG_DAYS}DMA_EURINR": dma_values
        })

        out.to_csv(output_file, mode="a", index=False, header=not header_written)
        header_written = True

    print(f"Output created: {output_file}")

print("\n ALL FILES PROCESSED SUCCESSFULLY")

Enter input PRICE folder path: E:\3Combinations_feb24_to_jan26\3_company_Total_Price
Enter FX (EURINR) file path: E:\3Combinations_feb24_to_jan26\EURINR_CLOSE.xlsx
Enter OUTPUT folder path: E:\3Combinations_feb24_to_jan26\3_company_EURINR_50DMA
Enter Moving Average days : 50
Total input CSV files detected: 86

Processing: Window_10_45Days.csv


Streaming: 13 rows [00:05,  2.34 rows/s]


Output created: E:\3Combinations_feb24_to_jan26\3_company_EURINR_50DMA\50DMA_Window_10_45Days.csv

Processing: Window_11_45Days.csv


Streaming: 13 rows [00:05,  2.29 rows/s]


Output created: E:\3Combinations_feb24_to_jan26\3_company_EURINR_50DMA\50DMA_Window_11_45Days.csv

Processing: Window_12_45Days.csv


...

Output created: E:\3Combinations_feb24_to_jan26\3_company_EURINR_50DMA\50DMA_Window_86_45Days.csv

Processing: Window_8_45Days.csv


Streaming: 13 rows [00:05,  2.39 rows/s]


Output created: E:\3Combinations_feb24_to_jan26\3_company_EURINR_50DMA\50DMA_Window_8_45Days.csv

Processing: Window_9_45Days.csv


Streaming: 13 rows [00:05,  2.40 rows/s]

Output created: E:\3Combinations_feb24_to_jan26\3_company_EURINR_50DMA\50DMA_Window_9_45Days.csv

 ALL FILES PROCESSED SUCCESSFULLY


In [1]:
#3Company_NIFTY_Ratio_50DMA

import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
import os
from collections import deque, defaultdict

# ====================== REQUIRED USER INPUTS ======================
INPUT_PRICE_FOLDER = input("Enter input PRICE folder path: ").strip()
INPUT_NIFTY_FILE = input("Enter NIFTY file path: ").strip()
OUTPUT_FOLDER = input("Enter OUTPUT folder path: ").strip()
MOVING_AVG_DAYS = int(input("Enter Moving Average days: "))

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====================== LOAD NIFTY DATA ======================
nifty = pd.read_excel(INPUT_NIFTY_FILE, usecols=["DATE", "NIFTY"])
nifty["DATE"] = pd.to_datetime(nifty["DATE"])
nifty.set_index("DATE", inplace=True)

# ====================== GET ALL INPUT FILES ======================
csv_files = sorted(glob.glob(os.path.join(INPUT_PRICE_FOLDER, "*.csv")))
print(f"Total input CSV files detected: {len(csv_files)}")

# ====================== PROCESS EACH FILE ======================
for file_csv in csv_files:

    print(f"\nProcessing: {os.path.basename(file_csv)}")

    output_file = os.path.join(
        OUTPUT_FOLDER,
        f"{MOVING_AVG_DAYS}DMA_NIFTY_{os.path.basename(file_csv)}"
    )

    # Rolling queue + sum per Combo
    ratio_queue = defaultdict(lambda: deque(maxlen=MOVING_AVG_DAYS))
    ratio_sum = defaultdict(float)

    header_written = False

    # Stream CSV in chunks
    for chunk in tqdm(
        pd.read_csv(
            file_csv,
            chunksize=50000,
            parse_dates=["DATE"],
            usecols=["DATE", "Combo", "Total_Price"]
        ),
        desc="Streaming",
        unit=" rows"
    ):
        # Merge NIFTY data
        chunk = chunk.join(nifty, on="DATE", how="left")
        chunk.dropna(subset=["NIFTY"], inplace=True)

        if chunk.empty:
            continue

        # Compute Ratio
        nifty_ratio = chunk["Total_Price"].values / chunk["NIFTY"].values
        dma_values = np.zeros(len(chunk))

        combos = chunk["Combo"].values

        # Rolling Moving Average Calculation
        for i, combo in enumerate(combos):
            q = ratio_queue[combo]

            # Maintain rolling sum
            if len(q) == MOVING_AVG_DAYS:
                ratio_sum[combo] -= q[0]

            q.append(nifty_ratio[i])
            ratio_sum[combo] += nifty_ratio[i]

            dma_values[i] = ratio_sum[combo] / len(q)

        # Prepare output chunk
        out = pd.DataFrame({
            "DATE": chunk["DATE"].values,
            "Combo": combos,
            "NIFTY_Ratio": nifty_ratio,
            f"{MOVING_AVG_DAYS}DMA_NIFTY": dma_values
        })

        out.to_csv(output_file, mode="a", index=False, header=not header_written)
        header_written = True

    print(f"Output saved: {output_file}")

print("\n ALL FILES PROCESSED SUCCESSFULLY.")

Enter input PRICE folder path: E:\3Combinations_feb24_to_jan26\3_company_Total_Price
Enter NIFTY file path: E:\3Combinations_feb24_to_jan26\NIFTY_CLOSE.xlsx
Enter OUTPUT folder path: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_50DMA
Enter Moving Average days: 50
Total input CSV files detected: 86

Processing: Window_10_45Days.csv


Streaming: 13 rows [00:05,  2.43 rows/s]


Output saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_50DMA\50DMA_NIFTY_Window_10_45Days.csv

Processing: Window_11_45Days.csv


Streaming: 13 rows [00:05,  2.34 rows/s]


Output saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_50DMA\50DMA_NIFTY_Window_11_45Days.csv

Processing: Window_12_45Days.csv


...

Output saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_50DMA\50DMA_NIFTY_Window_86_45Days.csv

Processing: Window_8_45Days.csv


Streaming: 13 rows [00:05,  2.45 rows/s]


Output saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_50DMA\50DMA_NIFTY_Window_8_45Days.csv

Processing: Window_9_45Days.csv


Streaming: 13 rows [00:05,  2.44 rows/s]

Output saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_50DMA\50DMA_NIFTY_Window_9_45Days.csv

 ALL FILES PROCESSED SUCCESSFULLY.


In [3]:
#NIFTY50_EURINR_Ratio > 1.01*50DMA_EURINR

import pandas as pd
import glob
import os

# ====================== USER INPUTS ======================
INPUT_FOLDER = input("Enter folder path of EURINR DMA CSV files: ").strip()
OUTPUT_FOLDER = input("Enter output folder path for filtered results: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====================== PROCESS FILES ======================
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"Total CSV files detected: {len(csv_files)}")

for file in csv_files:

    file_name = os.path.basename(file)
    print(f"\nProcessing: {file_name}")

    # Load file
    df = pd.read_csv(file)

    # Detect DMA column automatically (example: "50DMA_EURINR")
    dma_col = [col for col in df.columns if "DMA_EURINR" in col][0]

    # ================= APPLY CONDITION =================
    df_filtered = df[df["EURINR_Ratio"] > (1.01 * df[dma_col])].copy()

    # Count filtered rows
    filtered_count = len(df_filtered)
    print(f" Combinations satisfying condition: {filtered_count}")

    if df_filtered.empty:
        print("  No combinations satisfy condition → Skipped saving.")
        continue

    # Save file
    output_file = os.path.join(
        OUTPUT_FOLDER,
        f"Filtered_{file_name}"
    )

    df_filtered.to_csv(output_file, index=False)
    print(f" Saved filtered file: {output_file}")

print("\nALL FILES FILTERED SUCCESSFULLY.")


Enter folder path of EURINR DMA CSV files: E:\3Combinations_feb24_to_jan26\3_company_EURINR_50DMA
Enter output folder path for filtered results: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA
Total CSV files detected: 86

Processing: 50DMA_Window_10_45Days.csv
 Combinations satisfying condition: 150298
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_10_45Days.csv

Processing: 50DMA_Window_11_45Days.csv
 Combinations satisfying condition: 174297
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_11_45Days.csv

Processing: 50DMA_Window_12_45Days.csv
 Combinations satisfying condition: 177170
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_12_45Days.csv

Processing: 50DMA_Window_13_45Days.csv
 Combinations satisfying condition: 277528
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered

 Combinations satisfying condition: 287715
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_47_45Days.csv

Processing: 50DMA_Window_48_45Days.csv
 Combinations satisfying condition: 379167
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_48_45Days.csv

Processing: 50DMA_Window_49_45Days.csv
 Combinations satisfying condition: 321965
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_49_45Days.csv

Processing: 50DMA_Window_4_45Days.csv
 Combinations satisfying condition: 410243
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_4_45Days.csv

Processing: 50DMA_Window_50_45Days.csv
 Combinations satisfying condition: 238801
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_50_45Days.csv

Processing: 50DMA_Window_51_45Days.csv
 Combin

 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_85_45Days.csv

Processing: 50DMA_Window_86_45Days.csv
 Combinations satisfying condition: 2444
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_86_45Days.csv

Processing: 50DMA_Window_8_45Days.csv
 Combinations satisfying condition: 166569
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_8_45Days.csv

Processing: 50DMA_Window_9_45Days.csv
 Combinations satisfying condition: 139709
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA\Filtered_50DMA_Window_9_45Days.csv

ALL FILES FILTERED SUCCESSFULLY.


In [4]:
#NIFTY_Ratio > 1.01*50DMA_NIFTY

import pandas as pd
import glob
import os

# ====================== USER INPUTS ======================
INPUT_FOLDER = input("Enter folder path of NIFTY DMA CSV files: ").strip()
OUTPUT_FOLDER = input("Enter output folder path for filtered results: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====================== PROCESS FILES ======================
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"Total CSV files detected: {len(csv_files)}")

for file in csv_files:

    file_name = os.path.basename(file)
    print(f"\nProcessing: {file_name}")

    # Load file
    df = pd.read_csv(file)

    # Detect DMA column automatically (example: "50DMA_NIFTY")
    dma_col = [col for col in df.columns if "DMA_NIFTY" in col][0]

    # ================= APPLY CONDITION =================
    df_filtered = df[df["NIFTY_Ratio"] > (1.01 * df[dma_col])].copy()

    # Count filtered rows
    filtered_count = len(df_filtered)
    print(f" Combinations satisfying condition: {filtered_count}")

    if df_filtered.empty:
        print("  No combinations satisfy condition → Skipped saving.")
        continue

    # Save file
    output_file = os.path.join(
        OUTPUT_FOLDER,
        f"Filtered_{file_name}"
    )

    df_filtered.to_csv(output_file, index=False)
    print(f" Saved filtered file: {output_file}")

print("\nALL FILES FILTERED SUCCESSFULLY.")

Enter folder path of NIFTY DMA CSV files: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_50DMA
Enter output folder path for filtered results: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA
Total CSV files detected: 86

Processing: 50DMA_NIFTY_Window_10_45Days.csv
 Combinations satisfying condition: 169571
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_10_45Days.csv

Processing: 50DMA_NIFTY_Window_11_45Days.csv
 Combinations satisfying condition: 152291
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_11_45Days.csv

Processing: 50DMA_NIFTY_Window_12_45Days.csv
 Combinations satisfying condition: 140809
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_12_45Days.csv

Processing: 50DMA_NIFTY_Window_13_45Days.csv
 Combinations satisfying condition: 142661
 Saved filtered file: E:\3Combinations_feb24_to_jan2

 Combinations satisfying condition: 135191
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_45_45Days.csv

Processing: 50DMA_NIFTY_Window_46_45Days.csv
 Combinations satisfying condition: 124528
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_46_45Days.csv

Processing: 50DMA_NIFTY_Window_47_45Days.csv
 Combinations satisfying condition: 104387
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_47_45Days.csv

Processing: 50DMA_NIFTY_Window_48_45Days.csv
 Combinations satisfying condition: 104612
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_48_45Days.csv

Processing: 50DMA_NIFTY_Window_49_45Days.csv
 Combinations satisfying condition: 136251
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_49_45Days.

 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_80_45Days.csv

Processing: 50DMA_NIFTY_Window_81_45Days.csv
 Combinations satisfying condition: 87343
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_81_45Days.csv

Processing: 50DMA_NIFTY_Window_82_45Days.csv
 Combinations satisfying condition: 57115
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_82_45Days.csv

Processing: 50DMA_NIFTY_Window_83_45Days.csv
 Combinations satisfying condition: 34380
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_83_45Days.csv

Processing: 50DMA_NIFTY_Window_84_45Days.csv
 Combinations satisfying condition: 21758
 Saved filtered file: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA\Filtered_50DMA_NIFTY_Window_84_45Days.csv

Processing: 50DMA_NIFTY_Window_85_45Days.c

In [5]:
#EURINR_Ratio <= 1.03*50DMA

import os
import glob
import pandas as pd

# ================= USER INPUTS =================
INPUT_FOLDER  = input("Enter folder path with CSVs: ").strip()
OUTPUT_FOLDER = input("Enter output folder path: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Helper to create output CSV filename
def get_output_csv_name(input_csv):
    base = os.path.basename(input_csv)
    return os.path.join(OUTPUT_FOLDER, base)

# Load all CSV files
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"\nTotal CSV files detected: {len(csv_files)}")

for csv_file in csv_files:
    output_csv = get_output_csv_name(csv_file)
    print(f"\nProcessing {os.path.basename(csv_file)} ...")

    df = pd.read_csv(csv_file, parse_dates=["DATE"])

    # Required columns (EURINR only)
    required_cols = ["DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print(f"  Missing columns: {missing}. Skipping file.")
        continue

    # Apply EURINR condition only
    df_filtered = df[
        df["EURINR_Ratio"] <= 1.03 * df["50DMA_EURINR"]
    ]

    # Skip file if no rows satisfy the condition
    if df_filtered.empty:
        print("  No rows satisfy the EURINR condition. File skipped.")
        continue

    # Save filtered CSV
    df_filtered.to_csv(output_csv, index=False)
    print(f"  Filtered CSV saved: {output_csv} (rows: {len(df_filtered)})")

print("\nDONE! Only EURINR-based filtered combinations are saved.")


Enter folder path with CSVs: E:\3Combinations_feb24_to_jan26\3_company_EURINR_GT_50DMA
Enter output folder path: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA

Total CSV files detected: 86

Processing Filtered_50DMA_Window_10_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_10_45Days.csv (rows: 117962)

Processing Filtered_50DMA_Window_11_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_11_45Days.csv (rows: 140349)

Processing Filtered_50DMA_Window_12_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_12_45Days.csv (rows: 119078)

Processing Filtered_50DMA_Window_13_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_13_45Days.csv (rows: 182573)

Processing Filtered_50DMA_Window_14_45Days.csv ...
  Fil

  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_50_45Days.csv (rows: 148002)

Processing Filtered_50DMA_Window_51_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_51_45Days.csv (rows: 144077)

Processing Filtered_50DMA_Window_52_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_52_45Days.csv (rows: 126933)

Processing Filtered_50DMA_Window_53_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_53_45Days.csv (rows: 53886)

Processing Filtered_50DMA_Window_54_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA\Filtered_50DMA_Window_54_45Days.csv (rows: 85003)

Processing Filtered_50DMA_Window_55_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_5

In [6]:
#NIFTY_Ratio <= 1.03*50DMA

import os
import glob
import pandas as pd

# ================= USER INPUTS =================
INPUT_FOLDER  = input("Enter folder path with CSVs: ").strip()
OUTPUT_FOLDER = input("Enter output folder path: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Helper to create output CSV filename
def get_output_csv_name(input_csv):
    base = os.path.basename(input_csv)
    return os.path.join(OUTPUT_FOLDER, base)

# Load all CSV files
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"\nTotal CSV files detected: {len(csv_files)}")

for csv_file in csv_files:
    output_csv = get_output_csv_name(csv_file)
    print(f"\nProcessing {os.path.basename(csv_file)} ...")

    df = pd.read_csv(csv_file, parse_dates=["DATE"])

    # Required columns
    required_cols = ["DATE", "Combo", "NIFTY_Ratio", "50DMA_NIFTY"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print(f"  Missing columns: {missing}. Skipping file.")
        continue

    # Apply NIFTY condition only
    df_filtered = df[
        df["NIFTY_Ratio"] <= 1.03 * df["50DMA_NIFTY"]
    ]

    # Skip file if no rows satisfy the condition
    if df_filtered.empty:
        print("  No rows satisfy the NIFTY condition. File skipped.")
        continue

    # Save filtered CSV
    df_filtered.to_csv(output_csv, index=False)
    print(f"  Filtered CSV saved: {output_csv} (rows: {len(df_filtered)})")

print("\nDONE! Only NIFTY-based filtered combinations are saved.")


Enter folder path with CSVs: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_GT_50DMA
Enter output folder path: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA

Total CSV files detected: 86

Processing Filtered_50DMA_NIFTY_Window_10_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_10_45Days.csv (rows: 148461)

Processing Filtered_50DMA_NIFTY_Window_11_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_11_45Days.csv (rows: 131764)

Processing Filtered_50DMA_NIFTY_Window_12_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_12_45Days.csv (rows: 116830)

Processing Filtered_50DMA_NIFTY_Window_13_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_13_45Days.csv (rows: 116032)

Processing Fil

  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_48_45Days.csv (rows: 80283)

Processing Filtered_50DMA_NIFTY_Window_49_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_49_45Days.csv (rows: 101443)

Processing Filtered_50DMA_NIFTY_Window_4_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_4_45Days.csv (rows: 91107)

Processing Filtered_50DMA_NIFTY_Window_50_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_50_45Days.csv (rows: 100593)

Processing Filtered_50DMA_NIFTY_Window_51_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_51_45Days.csv (rows: 110362)

Processing Filtered_50DMA_NIFTY_Window_52_45Days.csv ...
  Filtered CSV saved: E:

  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_8_45Days.csv (rows: 140135)

Processing Filtered_50DMA_NIFTY_Window_9_45Days.csv ...
  Filtered CSV saved: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA\Filtered_50DMA_NIFTY_Window_9_45Days.csv (rows: 137450)

DONE! Only NIFTY-based filtered combinations are saved.


In [7]:
#common_combinations_EURINR_NIFTY

import pandas as pd
import glob
import os

# ====================== USER INPUTS ==========================
EURINR_FOLDER = input("Enter folder path of EURINR files: ").strip()
NIFTY_FOLDER  = input("Enter folder path of NIFTY files: ").strip()
OUTPUT_FOLDER = input("Enter output folder path: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

eur_files = sorted(glob.glob(os.path.join(EURINR_FOLDER, "*.csv")))
nifty_files = sorted(glob.glob(os.path.join(NIFTY_FOLDER, "*.csv")))

print(f"EURINR files found : {len(eur_files)}")
print(f"NIFTY  files found : {len(nifty_files)}")

def extract_month(file_name):
    """
    Example filename:
        50DMA_EURINR_Month_1_45Days.csv
        50DMA_NIFTY_Month_1_45Days.csv
    Function extracts: "Month_1_45Days"
    """
    parts = file_name.split("_")
    return "_".join(parts[-2:]).replace(".csv", "")   # Month_x_45Days

# Loop through all EURINR files
for eur_file in eur_files:

    eur_name = os.path.basename(eur_file)
    eur_month = extract_month(eur_name)

    # Find matching NIFTY file for SAME MONTH
    matching_nifty = [f for f in nifty_files if extract_month(os.path.basename(f)) == eur_month]

    if not matching_nifty:
        print(f"\n No NIFTY file found for: {eur_name}")
        continue

    nifty_file = matching_nifty[0]

    print(f"\n==========================")
    print(f" Processing Month: {eur_month}")
    print(f" EURINR ? {eur_name}")
    print(f" NIFTY  ? {os.path.basename(nifty_file)}")
    print(f"==========================")

    # Load files
    df_eur = pd.read_csv(eur_file)
    df_nifty = pd.read_csv(nifty_file)

    # Convert DATE to datetime
    df_eur["DATE"] = pd.to_datetime(df_eur["DATE"])
    df_nifty["DATE"] = pd.to_datetime(df_nifty["DATE"])

    # Select common combos within SAME MONTH files
    common_combos = sorted(set(df_eur["Combo"]).intersection(df_nifty["Combo"]))

    print(f" EURINR combos: {df_eur['Combo'].nunique()}")
    print(f" NIFTY combos : {df_nifty['Combo'].nunique()}")
    print(f" Common combos: {len(common_combos)}")

    if len(common_combos) == 0:
        print(" No common combinations ? Skipping this month.")
        continue

    # Filter EURINR and NIFTY to only common combos
    df_eur_f = df_eur[df_eur["Combo"].isin(common_combos)]
    df_nifty_f = df_nifty[df_nifty["Combo"].isin(common_combos)]

    # Merge on DATE + Combo ? ONE DATE column only
    df_final = df_eur_f.merge(
        df_nifty_f,
        on=["DATE", "Combo"],
        suffixes=("_EURINR", "_NIFTY")
    )

    # Save monthly output
    output_path = os.path.join(OUTPUT_FOLDER, f"Common_{eur_month}.csv")
    df_final.to_csv(output_path, index=False)

    print(f" Saved: {output_path}")

print("\n ALL MONTHS PROCESSED SUCCESSFULLY!")


Enter folder path of EURINR files: E:\3Combinations_feb24_to_jan26\3_company_EURINR_LTET_50DMA
Enter folder path of NIFTY files: E:\3Combinations_feb24_to_jan26\3_company_NIFTY_LTET_50DMA
Enter output folder path: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY
EURINR files found : 86
NIFTY  files found : 86

 Processing Month: 10_45Days
 EURINR ? Filtered_50DMA_Window_10_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_10_45Days.csv
 EURINR combos: 18926
 NIFTY combos : 17088
 Common combos: 16915
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_10_45Days.csv

 Processing Month: 11_45Days
 EURINR ? Filtered_50DMA_Window_11_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_11_45Days.csv
 EURINR combos: 19378
 NIFTY combos : 17474
 Common combos: 17359
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_11_45Days.csv

 Processing Month: 12_45Days
 EURINR ? Filtered_50DMA_Window_12_45

 EURINR combos: 15200
 NIFTY combos : 16291
 Common combos: 13492
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_30_45Days.csv

 Processing Month: 31_45Days
 EURINR ? Filtered_50DMA_Window_31_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_31_45Days.csv
 EURINR combos: 15021
 NIFTY combos : 17332
 Common combos: 14082
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_31_45Days.csv

 Processing Month: 32_45Days
 EURINR ? Filtered_50DMA_Window_32_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_32_45Days.csv
 EURINR combos: 16577
 NIFTY combos : 17983
 Common combos: 15864
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_32_45Days.csv

 Processing Month: 33_45Days
 EURINR ? Filtered_50DMA_Window_33_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_33_45Days.csv
 EURINR combos: 17746
 NIFTY combos : 17953
 Common combos: 16779
 Saved: E:\3Combinations_feb24_to_jan26\

 EURINR combos: 19353
 NIFTY combos : 14332
 Common combos: 14293
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_52_45Days.csv

 Processing Month: 53_45Days
 EURINR ? Filtered_50DMA_Window_53_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_53_45Days.csv
 EURINR combos: 15985
 NIFTY combos : 14594
 Common combos: 13181
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_53_45Days.csv

 Processing Month: 54_45Days
 EURINR ? Filtered_50DMA_Window_54_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_54_45Days.csv
 EURINR combos: 19325
 NIFTY combos : 14311
 Common combos: 14250
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_54_45Days.csv

 Processing Month: 55_45Days
 EURINR ? Filtered_50DMA_Window_55_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_55_45Days.csv
 EURINR combos: 18113
 NIFTY combos : 15236
 Common combos: 14886
 Saved: E:\3Combinations_feb24_to_jan26\

 EURINR combos: 18420
 NIFTY combos : 13937
 Common combos: 13937
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_74_45Days.csv

 Processing Month: 75_45Days
 EURINR ? Filtered_50DMA_Window_75_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_75_45Days.csv
 EURINR combos: 17809
 NIFTY combos : 13582
 Common combos: 13520
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_75_45Days.csv

 Processing Month: 76_45Days
 EURINR ? Filtered_50DMA_Window_76_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_76_45Days.csv
 EURINR combos: 14305
 NIFTY combos : 14015
 Common combos: 12913
 Saved: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY\Common_76_45Days.csv

 Processing Month: 77_45Days
 EURINR ? Filtered_50DMA_Window_77_45Days.csv
 NIFTY  ? Filtered_50DMA_NIFTY_Window_77_45Days.csv
 EURINR combos: 14196
 NIFTY combos : 13621
 Common combos: 12266
 Saved: E:\3Combinations_feb24_to_jan26\

In [8]:
#Top_25%_combinations_by_Average_Ratio

import os
import glob
import pandas as pd

# ================= USER INPUTS =================
INPUT_FOLDER  = input("Enter folder path with filtered CSVs: ").strip()
OUTPUT_FILE   = input("Enter output Excel file path: ").strip()
TOP_PERCENT   = float(input("Enter Top Percentage (example: 25 for top 25%): ").strip())

# Ensure valid % value
if TOP_PERCENT <= 0 or TOP_PERCENT > 100:
    print("Invalid percentage! Please enter value between 1 and 100.")
    exit()

# Ensure output folder exists
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

# Create safe Excel sheet names
def get_sheet_name(csv_file):
    name = os.path.basename(csv_file).replace(".csv", "")
    return name[:31]  

# Load all filtered CSV files
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"\nTotal filtered CSV files detected: {len(csv_files)}")

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:

    for csv_file in csv_files:
        print(f"\nProcessing {os.path.basename(csv_file)} ...")
        df = pd.read_csv(csv_file, parse_dates=["DATE"])

        required_cols = ["EURINR_Ratio", "NIFTY_Ratio", "Combo", "DATE"]
        missing = [c for c in required_cols if c not in df.columns]
        if missing:
            print(f" Missing columns {missing}. Skipping file.")
            continue

        # Internal ranking score 
        df["_Score"] = df["EURINR_Ratio"] + df["NIFTY_Ratio"]

        # Calculate count of top X%
        top_count = max(1, int((TOP_PERCENT / 100) * len(df)))

        # Select top rows
        df_top = df.nlargest(top_count, "_Score")

        # Remove internal column before saving
        df_top = df_top.drop(columns=["_Score"])

        # Write sheet
        sheet_name = get_sheet_name(csv_file)
        df_top.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"  Saved Top {TOP_PERCENT}% → Sheet: {sheet_name} (Rows: {len(df_top)})")

print("\nDONE!")
print(f"All Top {TOP_PERCENT}% combinations saved to:", OUTPUT_FILE)

Enter folder path with filtered CSVs: E:\3Combinations_feb24_to_jan26\3_company_common_combinations_EURINR_NIFTY
Enter output Excel file path: E:\3Combinations_feb24_to_jan26\3_company_Top_25%_combinations_by_Average_Ratio.xlsx
Enter Top Percentage (example: 25 for top 25%): 25

Total filtered CSV files detected: 86

Processing Common_10_45Days.csv ...
  Saved Top 25.0% → Sheet: Common_10_45Days (Rows: 13582)

Processing Common_11_45Days.csv ...
  Saved Top 25.0% → Sheet: Common_11_45Days (Rows: 13873)

Processing Common_12_45Days.csv ...
  Saved Top 25.0% → Sheet: Common_12_45Days (Rows: 13137)

Processing Common_13_45Days.csv ...
  Saved Top 25.0% → Sheet: Common_13_45Days (Rows: 13491)

Processing Common_14_45Days.csv ...
  Saved Top 25.0% → Sheet: Common_14_45Days (Rows: 14416)

Processing Common_15_45Days.csv ...
  Saved Top 25.0% → Sheet: Common_15_45Days (Rows: 13830)

Processing Common_16_45Days.csv ...
  Saved Top 25.0% → Sheet: Common_16_45Days (Rows: 15500)

Processing Commo

  Saved Top 25.0% → Sheet: Common_8_45Days (Rows: 14416)

Processing Common_9_45Days.csv ...
  Saved Top 25.0% → Sheet: Common_9_45Days (Rows: 10819)

DONE!
All Top 25.0% combinations saved to: E:\3Combinations_feb24_to_jan26\3_company_Top_25%_combinations_by_Average_Ratio.xlsx


In [9]:
#Lowest_Average_Realized_Correlation(4Stock_Combinations_Correlation)

import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm

# ================= USER INPUT =================
input_file = r"E:\3Combinations_feb24_to_jan26\NIFTY50_CLOSE_MASTER.xlsx"
output_file = r"E:\3Combinations_feb24_to_jan26\3Stock_Combinations_Correlation_input_data.xlsx"


print("Loading data...")
df = pd.read_excel(input_file)

# Convert DATE column
df['DATE'] = pd.to_datetime(df['DATE'])

# Set DATE as index
df.set_index('DATE', inplace=True)

# Calculate daily returns
returns = df.pct_change().dropna()

stocks = returns.columns.tolist()
print(f"Total Stocks: {len(stocks)}")

results = []

# Generate all 3-stock combinations
combinations = list(itertools.combinations(stocks, 3))
print(f"Total 3-stock combinations: {len(combinations)}")

for combo in tqdm(combinations, desc="Calculating correlations"):

    data = returns[list(combo)]
    corr_matrix = data.corr()

    # All 6 pairwise correlations
    pairs = list(itertools.combinations(combo, 2))
    pair_corrs = [corr_matrix.loc[a, b] for a, b in pairs]

    #  YOUR EXACT FORMULA
    avg_realized_corr = (0.25 * sum(pair_corrs)) / 1.5

    results.append({
        "Combo": ", ".join(combo),
        "Average_Realized_Correlation": avg_realized_corr
    })

# Create result dataframe
result_df = pd.DataFrame(results)

# Rank by least correlation
result_df = result_df.sort_values(by="Average_Realized_Correlation")

# Save output
result_df.to_excel(output_file, index=False)

print(" DONE")
print(" The TOP row in Excel is your LEAST CORRELATED 3-stock portfolio")


Loading data...


C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13068\1960523217.py:23: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df.pct_change().dropna()


Total Stocks: 50
Total 3-stock combinations: 19600


Calculating correlations: 100%|████████████████████████████████████████████████| 19600/19600 [00:12<00:00, 1546.62it/s]


 DONE
 The TOP row in Excel is your LEAST CORRELATED 3-stock portfolio


In [10]:
#Top_25%_Lowest_Average_Correlation

import pandas as pd
import os

# ---------------- USER INPUT ----------------
file_corr = input("Enter the path of the Correlation Excel file: ").strip()
file_data = input("Enter the path of the data Excel file (multiple sheets): ").strip()
output_file = input("Enter the path for the output Excel file: ").strip()

# Ensure output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

def normalize_combo(combo):
    if pd.isna(combo):
        return combo
    stocks = [s.strip() for s in combo.split(",")]
    stocks_sorted = sorted(stocks)
    return ", ".join(stocks_sorted)

print("Loading correlation file...")
df_corr = pd.read_excel(file_corr)
df_corr["Combo_Normalized"] = df_corr["Combo"].apply(normalize_combo)

print("Loading data file with multiple sheets...")
xls_data = pd.ExcelFile(file_data)
sheet_names = xls_data.sheet_names

output_sheets = {}

for sheet in sheet_names:
    print(f"Processing sheet: {sheet} ...")
    df_data = pd.read_excel(xls_data, sheet_name=sheet)
    
    # Normalize combo
    df_data["Combo_Normalized"] = df_data["Combo"].apply(normalize_combo)
    
    # Merge correlation
    df_merged = df_data.merge(
        df_corr[["Combo_Normalized", "Average_Realized_Correlation"]],
        on="Combo_Normalized",
        how="left"
    )
    
    # Drop rows without correlation if needed
    df_merged = df_merged.dropna(subset=["Average_Realized_Correlation"])
    
    # Remove helper column
    df_merged.drop(columns=["Combo_Normalized"], inplace=True)
    
    output_sheets[sheet] = df_merged

print("Saving output file with multiple sheets...")
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet, df_out in output_sheets.items():
        df_out.to_excel(writer, sheet_name=sheet, index=False)

print("DONE: Correlation values applied and output saved with multiple sheets.")

Enter the path of the Correlation Excel file: E:\3Combinations_feb24_to_jan26\3Stock_Combinations_Correlation_input_data.xlsx
Enter the path of the data Excel file (multiple sheets): E:\3Combinations_feb24_to_jan26\3_company_Top_25%_combinations_by_Average_Ratio.xlsx
Enter the path for the output Excel file: E:\3Combinations_feb24_to_jan26\3_company_Top_25%_Lowest_Average_Correlation.xlsx
Loading correlation file...
Loading data file with multiple sheets...
Processing sheet: Common_10_45Days ...
Processing sheet: Common_11_45Days ...
Processing sheet: Common_12_45Days ...
Processing sheet: Common_13_45Days ...
Processing sheet: Common_14_45Days ...
Processing sheet: Common_15_45Days ...
Processing sheet: Common_16_45Days ...
Processing sheet: Common_17_45Days ...
Processing sheet: Common_18_45Days ...
Processing sheet: Common_19_45Days ...
Processing sheet: Common_1_45Days ...
Processing sheet: Common_20_45Days ...
Processing sheet: Common_21_45Days ...
Processing sheet: Common_22_45Da

In [1]:
#Correlation_with_Japan_Bond_Yield( for each Combo)

import pandas as pd

# ---------------- USER INPUT ----------------
file_prices = input("Enter the path of the Prices CSV file: ").strip()
file_japan  = input("Enter the path of the Japan Yield Excel file: ").strip()
output_file = input("Enter the path for the output CSV file: ").strip()

# ---------------- LOAD REQUIRED COLUMNS ----------------
df_prices = pd.read_csv(
    file_prices,
    usecols=['DATE', 'Combo', 'Total_Price']
)

df_japan = pd.read_excel(
    file_japan,
    usecols=['DATE', 'Japan_Yield']
)

# ---------------- DATE CONVERSION ----------------
df_prices['DATE'] = pd.to_datetime(df_prices['DATE'], errors='coerce')
df_japan['DATE']  = pd.to_datetime(df_japan['DATE'], errors='coerce')

df_prices.dropna(subset=['DATE'], inplace=True)
df_japan.dropna(subset=['DATE'], inplace=True)

# ---------------- SORT & MERGE ----------------
df_prices.sort_values('DATE', inplace=True)
df_japan.sort_values('DATE', inplace=True)

merged = pd.merge(df_prices, df_japan, on='DATE', how='inner')

# ---------------- CORRELATION PER COMBO ----------------
results = (
    merged
    .groupby('Combo')[['Total_Price', 'Japan_Yield']]
    .corr()
    .iloc[0::2, -1]   
    .reset_index(level=1, drop=True)
    .reset_index(name='Correlation_with_Japan_Yield')
)

# ---------------- SAVE OUTPUT ----------------
results.to_csv(output_file, index=False)

print("\n Correlation calculation completed successfully.")
print(f" Output saved to: {output_file}")


Enter the path of the Prices CSV file: E:\3Combinations_feb24_to_jan26\3_company_Total_Price\Window_1_45Days.csv
Enter the path of the Japan Yield Excel file: D:\feb24_to_jan26\Japan 10-Year Bond Yield Historical  Data.xlsx
Enter the path for the output CSV file: E:\3Combinations_feb24_to_jan26\3_company_Correlation_with_Japan_Bond_Yield.csv

 Correlation calculation completed successfully.
 Output saved to: E:\3Combinations_feb24_to_jan26\3_company_Correlation_with_Japan_Bond_Yield.csv


In [2]:
#NIFTY50_Top_25%_Highest_Correlation_Japan_Bond_Yield

import pandas as pd
import os

# ---------------- USER INPUT ----------------
file_corr = input("Enter the path of the Correlation CSV file: ").strip()
file_data = input("Enter the path of the data Excel file (multiple sheets): ").strip()
output_file = input("Enter the path for the output Excel file: ").strip()

# Ensure output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# -------- Normalize combo sorting -------
def normalize_combo(combo):
    if pd.isna(combo):
        return combo
    stocks = [s.strip() for s in combo.split(",")]
    return ", ".join(sorted(stocks))

# -------- Load correlation CSV --------
print("Loading correlation CSV file...")
df_corr = pd.read_csv(file_corr)
df_corr["Combo_Normalized"] = df_corr["Combo"].apply(normalize_combo)

# -------- Take highest Japan Yield correlation --------
df_corr_grouped = (
    df_corr.groupby("Combo_Normalized", as_index=False)["Correlation_with_Japan_Yield"]
           .max()
)

# -------- Load multi-sheet Excel --------
print("Loading data file with multiple sheets...")
xls_data = pd.ExcelFile(file_data)
sheet_names = xls_data.sheet_names

output_sheets = {}

# -------- Process each sheet --------
for sheet in sheet_names:
    print(f"Processing sheet: {sheet} ...")
    df_data = pd.read_excel(xls_data, sheet_name=sheet)

    # Normalize combo in data
    df_data["Combo_Normalized"] = df_data["Combo"].apply(normalize_combo)

    # Merge highest correlation
    df_merged = df_data.merge(
        df_corr_grouped,
        on="Combo_Normalized",
        how="left"
    )

    # Rename output column
    df_merged.rename(
        columns={"Correlation_with_Japan_Yield": "Highest_Correlation_Japan_Bond_Yield"},
        inplace=True
    )

    # Remove rows where correlation not available
    df_merged = df_merged.dropna(subset=["Highest_Correlation_Japan_Bond_Yield"])

    # Remove helper column
    df_merged.drop(columns=["Combo_Normalized"], inplace=True)

    output_sheets[sheet] = df_merged

# -------- Save Excel with multiple sheets --------
print("Saving output file with multiple sheets...")
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet, df_out in output_sheets.items():
        df_out.to_excel(writer, sheet_name=sheet, index=False)

print("DONE: Output Excel generated with all sheets updated.")

Enter the path of the Correlation CSV file: E:\3Combinations_feb24_to_jan26\3_company_Correlation_with_Japan_Bond_Yield.csv
Enter the path of the data Excel file (multiple sheets): E:\3Combinations_feb24_to_jan26\3_company_Top_25%_Lowest_Average_Correlation.xlsx
Enter the path for the output Excel file: E:\3Combinations_feb24_to_jan26\3_company_Top_25%_Highest_Correlation_Japan_Bond_Yield.xlsx
Loading correlation CSV file...
Loading data file with multiple sheets...
Processing sheet: Common_10_45Days ...
Processing sheet: Common_11_45Days ...
Processing sheet: Common_12_45Days ...
Processing sheet: Common_13_45Days ...
Processing sheet: Common_14_45Days ...
Processing sheet: Common_15_45Days ...
Processing sheet: Common_16_45Days ...
Processing sheet: Common_17_45Days ...
Processing sheet: Common_18_45Days ...
Processing sheet: Common_19_45Days ...
Processing sheet: Common_1_45Days ...
Processing sheet: Common_20_45Days ...
Processing sheet: Common_21_45Days ...
Processing sheet: Commo

In [1]:
#Top_10_Combo_Highest_Correlation_Japan_Bond_Yield

import pandas as pd
import os

# ---------------- USER INPUT ----------------
input_excel = input("Enter the input Excel file path (multiple sheets): ").strip()
output_excel = input("Enter the output Excel file path: ").strip()

# Ensure output directory exists
os.makedirs(os.path.dirname(output_excel), exist_ok=True)

REQUIRED_COL = "Highest_Correlation_Japan_Bond_Yield"
COMBO_COL = "Combo"
TOP_N = 10

print("Loading input Excel file...")
xls = pd.ExcelFile(input_excel)

valid_sheet_count = 0

with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    for sheet in xls.sheet_names:
        print(f"\nProcessing sheet: {sheet}")

        df = pd.read_excel(xls, sheet_name=sheet)

        # ---- Column validation ----
        if REQUIRED_COL not in df.columns or COMBO_COL not in df.columns:
            print(f" Skipped → Missing required columns")
            continue

        # ---- Sort & deduplicate ----
        df_sorted = (
            df.sort_values(REQUIRED_COL, ascending=False)
              .drop_duplicates(subset=COMBO_COL, keep="first")
        )

        # ---- Check top 10 availability ----
        if len(df_sorted) < TOP_N:
            print(f" Skipped → Only {len(df_sorted)} unique combos (need {TOP_N})")
            continue

        # ---- Take top 10 ----
        df_top = df_sorted.head(TOP_N)

        # ---- Write sheet ----
        df_top.to_excel(writer, sheet_name=sheet, index=False)
        valid_sheet_count += 1
        print(" Sheet written")

# ---- Final status ----
if valid_sheet_count == 0:
    print("\n No sheets met the criteria. Output file may be empty.")
else:
    print(f"\n DONE: {valid_sheet_count} valid sheets written successfully.")

Enter the input Excel file path (multiple sheets): D:\3Combinations_feb24_to_jan26\3_company_Top_25%_Highest_Correlation_Japan_Bond_Yield.xlsx
Enter the output Excel file path: D:\3Combinations_feb24_to_jan26\3_company_Top_10_Combo_Highest_Correlation_Japan_Bond_Yield.xlsx
Loading input Excel file...

Processing sheet: Common_10_45Days
 Sheet written

Processing sheet: Common_11_45Days
 Sheet written

Processing sheet: Common_12_45Days
 Sheet written

Processing sheet: Common_13_45Days
 Sheet written

Processing sheet: Common_14_45Days
 Sheet written

Processing sheet: Common_15_45Days
 Sheet written

Processing sheet: Common_16_45Days
 Sheet written

Processing sheet: Common_17_45Days
 Sheet written

Processing sheet: Common_18_45Days
 Sheet written

Processing sheet: Common_19_45Days
 Sheet written

Processing sheet: Common_1_45Days
 Sheet written

Processing sheet: Common_20_45Days
 Sheet written

Processing sheet: Common_21_45Days
 Sheet written

Processing sheet: Common_22_45Days


In [2]:
#only_select_combos

import pandas as pd
import os
import re

# ================= USER INPUT =================
INPUT_FILE  = r"D:\3Combinations_feb24_to_jan26\3_company_Top_10_Combo_Highest_Correlation_Japan_Bond_Yield.xlsx"
OUTPUT_FILE = r"D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\only_select_10_combos.xlsx"

# ================= LOAD ALL SHEETS =================
xls = pd.ExcelFile(INPUT_FILE)

os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

def extract_number(sheet_name):
    """
    Extracts the numeric part from Common_<number>_45Days
    """
    match = re.search(r"Common_(\d+)_", sheet_name)
    return int(match.group(1)) if match else float("inf")

# Sort sheet names numerically
sorted_sheets = sorted(xls.sheet_names, key=extract_number)

with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:
    for sheet_name in sorted_sheets:
        df = pd.read_excel(xls, sheet_name=sheet_name)

        # Keep ONLY Combo column
        out_df = df[["Combo"]] if "Combo" in df.columns else pd.DataFrame({"Combo": []})

        # Write with SAME sheet name
        out_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Output file created with sheets ordered serially (numeric), names unchanged.")

Output file created with sheets ordered serially (numeric), names unchanged.


In [3]:
#Combo_With_other_columns

import pandas as pd

# ================= FILE PATHS =================
FIRST_FILE  = r"D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\Input_date_data_for_performance.xlsx"       
SECOND_FILE = r"D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\only_select_10_combos.xlsx"
OUTPUT_FILE = r"D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\10_Combo_With_dates_columns.xlsx"

# ================= LOAD FILES =================
first_xls  = pd.ExcelFile(FIRST_FILE)
second_xls = pd.ExcelFile(SECOND_FILE)

with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:

    for i, sheet_name in enumerate(second_xls.sheet_names):

        # Read files
        first_df  = pd.read_excel(first_xls, sheet_name=first_xls.sheet_names[i])
        second_df = pd.read_excel(second_xls, sheet_name=sheet_name)

        # Identify column names safely
        col2_name = first_df.columns[1]
        col3_name = first_df.columns[2]

        # Keep only required columns from first file
        lookup_df = first_df[["Combo", col2_name, col3_name]]

        # ================= COMBO-BASED MERGE =================
        final_df = second_df.merge(
            lookup_df,
            on="Combo",
            how="left"
        )

        # Write output
        final_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(" Error fixed: columns added using Combo matching.")

 Error fixed: columns added using Combo matching.


In [4]:
#Combo_Top_10_total_price

import os
import pandas as pd
import re
import numpy as np

# ================= USER INPUT =================
COMBO_FILE  = input("Enter Combo Excel file path: ").strip()
PRICE_FILE  = input("Enter Price Excel file path: ").strip()
OUTPUT_FILE = input("Enter Output Excel file path: ").strip()
INVESTMENT  = float(input("Enter investment per company (e.g. 50000): "))

# ================= CLEAN OLD OUTPUT =================
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)

# ================= LOAD PRICE DATA =================
price_df = pd.read_excel(PRICE_FILE)
price_df["DATE"] = pd.to_datetime(price_df["DATE"], format="%d-%b-%y")
price_df.set_index("DATE", inplace=True)
price_index = price_df.index

# ================= HELPER FUNCTIONS =================
def extract_date(col_name):
    match = re.search(r"\d{4}-\d{2}-\d{2}", col_name)
    if not match:
        raise ValueError(f"Date not found in column name: {col_name}")
    return pd.to_datetime(match.group())

def nearest_date(index, target):
    return index[index.get_indexer([target], method="nearest")[0]]

# ================= PROCESS COMBO FILE =================
combo_xls = pd.ExcelFile(COMBO_FILE)

with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:

    for sheet_name in combo_xls.sheet_names:

        combo_df = pd.read_excel(combo_xls, sheet_name=sheet_name)

        # ---- Extract dates from column names ----
        start_date = extract_date(combo_df.columns[1])
        end_date   = extract_date(combo_df.columns[2])

        if start_date not in price_index:
            start_date = nearest_date(price_index, start_date)
        if end_date not in price_index:
            end_date = nearest_date(price_index, end_date)

        start_prices = price_df.loc[start_date]
        end_prices   = price_df.loc[end_date]

        output_rows = []

        for combo in combo_df["Combo"]:
            companies = combo.split(",")

            # ---- Quantity calculation ----
            quantities = INVESTMENT / start_prices[companies].values

            start_total = int(round(INVESTMENT * len(companies)))
            end_total   = int(round(np.sum(quantities * end_prices[companies].values)))

            output_rows.append({
                "Combo": combo,
                "start_price": start_total,
                "final_price": end_total
            })

        out_df = pd.DataFrame(output_rows)

        #  OUTPUT SHEET NAME = INPUT SHEET NAME
        out_df.to_excel(writer, sheet_name=sheet_name[:31], index=False)

print("\n Process completed successfully")
print(f" Output file created at: {OUTPUT_FILE}")

Enter Combo Excel file path: D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\10_Combo_With_dates_columns.xlsx
Enter Price Excel file path: D:\3Combinations_feb24_to_jan26\NIFTY50_CLOSE_MASTER.xlsx
Enter Output Excel file path: D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\Combo_Top_10_total_price.xlsx
Enter investment per company (e.g. 50000): 90000

 Process completed successfully
 Output file created at: D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\Combo_Top_10_total_price.xlsx


In [5]:
# Combo_Top_10_Gain_Loss

import pandas as pd
import numpy as np

# ===================== USER INPUT =====================
input_file = input("Enter input Excel file path: ").strip()
output_file = input("Enter output Excel file path: ").strip()

# ===================== LOAD INPUT =====================
xls = pd.ExcelFile(input_file)

summary_rows = []
detailed_sheets = {}

# ===================== PROCESS EACH INPUT SHEET =====================
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)

    # -------- FIND PRICE COLUMNS DYNAMICALLY --------
    start_col = next((c for c in df.columns if c.startswith("start_price")), None)
    final_col = next((c for c in df.columns if c.startswith("final_price")), None)

    if start_col is None or final_col is None:
        raise KeyError(
            f"Sheet '{sheet_name}' missing start_price or final_price column"
        )

    # ----- ADD CALCULATION COLUMNS -----
    df["Gain_Loss_Amt"] = df[final_col] - df[start_col]
    df["Gain_Amt"] = np.where(df["Gain_Loss_Amt"] > 0, df["Gain_Loss_Amt"], 0)
    df["Loss_Amt"] = np.where(df["Gain_Loss_Amt"] < 0, abs(df["Gain_Loss_Amt"]), 0)
    df["No_of_Gains"] = np.where(df["Gain_Loss_Amt"] > 0, 1, 0)
    df["No_of_Loss"] = np.where(df["Gain_Loss_Amt"] < 0, 1, 0)

    # ----- SELECT COLUMNS FOR DETAILED SHEET -----
    detailed_df = df[
        ["Combo", start_col, final_col, "Gain_Loss_Amt", "Gain_Amt", "Loss_Amt"]
    ]

    detailed_sheets[sheet_name] = detailed_df

    # ----- CREATE SHEET-WISE SUMMARY -----
    summary_rows.append({
        "Sheet_Name": sheet_name,
        "No_of_Gains": int(df["No_of_Gains"].sum()),
        "No_of_Loss": int(df["No_of_Loss"].sum()),
        "Total_Gains_Amt": round(df["Gain_Amt"].sum(), 2),
        "Total_Loss_Amt": round(df["Loss_Amt"].sum(), 2),
        "Net_Gain_Loss_Amt": round(df["Gain_Amt"].sum() - df["Loss_Amt"].sum(), 2)
    })

# ===================== WRITE OUTPUT =====================
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:

    # Summary first
    summary_df = pd.DataFrame(summary_rows)
    summary_df.to_excel(writer, sheet_name="Summary_All_Sheets", index=False)

    # Detailed sheets
    for sheet_name, df in detailed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(" Output created successfully")

Enter input Excel file path: D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\Combo_Top_10_total_price.xlsx
Enter output Excel file path: D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\Combo_Top_10_Gain_Loss.xlsx
 Output created successfully


In [7]:
#Summary_With_Percentage_Columns

import pandas as pd

# ================= USER INPUT =================
input_file = input("Enter input Excel file path: ").strip()
output_file = input("Enter output Excel file path: ").strip()

# ================= LOAD INPUT  =================
df = pd.read_excel(input_file)

# ================= CONSTANTS =================
CAPITAL = 270000 * 10
TOTAL_COMBOS = 10
TOTAL_WEEKS = len(df)   

# ================= ADD OUTPUT COLUMNS =================
df["%Returns"] = ((df["Net_Gain_Loss_Amt"] / CAPITAL) * 100).round(2)
df["%No_of_Gains"] = ((df["No_of_Gains"] / TOTAL_COMBOS) * 100).round(2)

df["%No_of_Loss"] = ((df["No_of_Loss"] / TOTAL_COMBOS) * 100).round(2)

# ================= OVERALL SUMMARY =================
overall_df = pd.DataFrame({
    "Metric": [
        "%Total_Returns",
        "%Total_No_of_Gains",
        "%Total_No_of_Loss"
    ],
    "Value": [
        round((df["Net_Gain_Loss_Amt"].sum() / (CAPITAL * TOTAL_WEEKS)) * 100,2),
        round((df["No_of_Gains"].sum() / (TOTAL_COMBOS * TOTAL_WEEKS)) * 100,2),
        round((df["No_of_Loss"].sum() / (TOTAL_COMBOS * TOTAL_WEEKS)) * 100,2)
    ]
})

# ================= WRITE OUTPUT =================
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="Weekly_Data", index=False)
    overall_df.to_excel(writer, sheet_name="Overall_Summary", index=False)

print("\n Calculation completed successfully.")
print(" Output file created at:", output_file)

Enter input Excel file path: D:\3Combinations_feb24_to_jan26\Performance_gain_loss\Top_10\only_summary_10.xlsx
Enter output Excel file path: D:\3Combinations_feb24_to_jan26\Performance_gain_loss\3_stocks_Summary_86weeks.xlsx

 Calculation completed successfully.
 Output file created at: D:\3Combinations_feb24_to_jan26\Performance_gain_loss\3_stocks_Summary_86weeks.xlsx
